In [1]:
import os
import glob
import sys
from lat_lon_parser import parse
import geopandas as gpd
import pandas as pd
import exif
import pillow_heif
import numpy as np

import folium
from folium.features import CustomIcon

from PIL import Image

In [2]:
# define fucntion to get file exif information
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image.getexif().get_ifd(0x8825)

# define fucntion to get location data from exif
def get_geotagging(exif):
    geo_tagging_info = {}
    # If no file image info return location data as (N:0, W:0)
    if not exif:
        return {'GPSLatitudeRef': 'N',
                'GPSLatitude': '(0, 0, 0)',
                'GPSLongitudeRef': 'W',
                'GPSLongitude': '(0, 0, 0)'}
    else:
        gps_keys = ['GPSVersionID', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude',
                    'GPSAltitudeRef', 'GPSAltitude', 'GPSTimeStamp', 'GPSSatellites', 'GPSStatus', 'GPSMeasureMode',
                    'GPSDOP', 'GPSSpeedRef', 'GPSSpeed', 'GPSTrackRef', 'GPSTrack', 'GPSImgDirectionRef',
                    'GPSImgDirection', 'GPSMapDatum', 'GPSDestLatitudeRef', 'GPSDestLatitude', 'GPSDestLongitudeRef',
                    'GPSDestLongitude', 'GPSDestBearingRef', 'GPSDestBearing', 'GPSDestDistanceRef', 'GPSDestDistance',
                    'GPSProcessingMethod', 'GPSAreaInformation', 'GPSDateStamp', 'GPSDifferential']

        for k, v in exif.items():
            try:
                geo_tagging_info[gps_keys[k]] = str(v)
            except IndexError:
                pass
        return geo_tagging_info

In [3]:
path = '/Users/fran/Downloads/Peel_park/'
# cerate empty df
master_df =pd.DataFrame()
paths =os.listdir(path)

# For sub-directory in the full directory
for p in paths:
      if p.endswith(".jpeg"):

            image_info = get_exif(path+p)
            #print(image_info)
            results = get_geotagging(image_info)
            #print(results)
            df=pd.DataFrame(columns =['IMGname','Latitude','Longitude'])
            
            df.loc[0]= [p.rstrip(".jpeg"),parse(results['GPSLatitude']),(parse(results['GPSLongitude'])*-1) ]
            df["file_path"] = str(path+p)
            master_df =pd.concat([master_df,df])  

In [4]:
master_df

,IMGname,Latitude,Longitude,file_path
0,IMG_7820,53.810925,-1.739292,/Users/fran/Downloads/Peel_park/IMG_7820.jpeg
0,IMG_7877,53.807831,-1.745608,/Users/fran/Downloads/Peel_park/IMG_7877.jpeg
0,IMG_7898,53.809106,-1.738875,/Users/fran/Downloads/Peel_park/IMG_7898.jpeg
0,IMG_7861,53.809769,-1.745086,/Users/fran/Downloads/Peel_park/IMG_7861.jpeg
0,IMG_7836,53.809958,-1.742394,/Users/fran/Downloads/Peel_park/IMG_7836.jpeg
...,...,...,...,...
0,IMG_7874,53.807869,-1.746036,/Users/fran/Downloads/Peel_park/IMG_7874.jpeg
0,IMG_7862,53.809769,-1.745097,/Users/fran/Downloads/Peel_park/IMG_7862.jpeg
0,IMG_7835,53.809958,-1.742394,/Users/fran/Downloads/Peel_park/IMG_7835.jpeg
0,IMG_7819,53.810933,-1.739267,/Users/fran/Downloads/Peel_park/IMG_7819.jpeg


In [19]:
peel_park_image_locations = gpd.GeoDataFrame(master_df, geometry=gpd.points_from_xy(master_df.Longitude, master_df.Latitude), crs="EPSG:4326")

In [20]:
peel_park_image_locations =peel_park_image_locations.replace(0, np.nan)

In [21]:
peel_park_image_locations.loc[peel_park_image_locations['Latitude']==0,:]

,IMGname,Latitude,Longitude,file_path,geometry


In [23]:
peel_park_image_locations.explore()

In [22]:
peel_park_image_locations.to_file('peel_park_image_locations.geojson')

In [13]:


# Load your GeoDataFrame
gdf = peel_park_image_locations


# Define the center of the map
latitude = 53.8085 # Example latitude (London)
longitude = -1.7437 # Example longitude (London)


# Create a Folium map
m = folium.Map(location=[latitude, longitude], zoom_start=10)

# Function to add markers with tooltips
def add_markers_with_tooltips(gdf, map_obj):
    for _, row in gdf.iterrows():
        # Create a custom icon with an image
        icon = CustomIcon(icon_image=row['file_path'], icon_size=(50, 50))
        
        # Create a marker with a tooltip
        marker = folium.Marker(
            location=[row['geometry'].y, row['geometry'].x],
            icon=icon,
            tooltip=folium.Tooltip(f"<img src='{row['file_path']}' width='100' height='100'><br>{row['IMGname']}")
        )
        marker.add_to(map_obj)

# Add markers to the map
add_markers_with_tooltips(gdf, m)

# Save the map to an HTML file
m.save('map_with_tooltips.html')


In [14]:
m